In [1]:
import torch
import utils
import time
from torch import nn
from torch.utils import data
from concreteNet import ConcreteAutoencoder

/home/qinxianhan/.conda/envs/singleCell/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_path = "../data/filtered_gene_bc_matrices/hg19"
device = utils.get_device()

In [3]:
adata = utils.anndata_load(file_path)
adata = utils.anndata_preprocess(adata)

In [4]:
adata.shape

(2700, 13714)

In [5]:
def train(net, loss, dataloader, num_epoch, learning_rate, weight_decay, device):
    net = net.to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate, weight_decay=weight_decay)
    start_time  = time.time()
    net.train()
    for epoch in range(num_epoch):
        train_loss = 0
        for data in dataloader:
            input = data.X.to(device, non_block = True)
            optimizer.zero_grad()
            l = loss(net(input), input)
            l.backward()
            optimizer.step()
            train_loss += l
        process_time = time.time() - start_time
        train_loss /= len(dataloader)
        print(f"Epoch: {epoch}; Loss {train_loss}; Time: {process_time} s")


In [6]:
def get_net(input_dim, hidden_dim, k, temperature):
    return ConcreteAutoencoder(input_dim, hidden_dim, k, temperature)
def load_adata(adata, batch_size):
    return utils.data_loader(adata, batch_size)

In [ ]:
num_epoch, learning_rate, weight_dacay, batch_size, temperature = 128, 0.1, 0, 128, 0.1
input_dim = adata.n_vars
loss = nn.MSELoss()
net = get_net(input_dim=input_dim, hidden_dim=128, k=1000, temperature=temperature)

In [ ]:
dataloader=load_adata(adata=adata, batch_size=batch_size)

In [ ]:
train(net)